<a href="https://colab.research.google.com/github/HimanshuGitCode/Advanced-AI-Program/blob/main/First_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
from typing import Optional, Dict, Any, List
from openai import OpenAI
import yfinance as yf
# from dotenv import load_dotenv  # Remove dotenv
# import os # Keep os if needed for other things, but remove if only for dotenv

# Load environment variables - No longer needed with userdata
# load_dotenv()

from google.colab import userdata # Import userdata

class StockInfoAgent:
    def __init__(self):
        # self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY')) # Use userdata instead
        self.client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
        self.conversation_history = []

    def get_stock_price(self, ticker_symbol: str) -> Optional[str]:
        """Fetches the current stock price for the given ticker_symbol."""
        try:
            stock = yf.Ticker(ticker_symbol.upper())
            info = stock.info
            current_price = info.get('currentPrice') or info.get('regularMarketPrice')

            if current_price:
                return f"{current_price:.2f} USD"
            return None
        except Exception as e:
            print(f"Error fetching stock price: {e}")
            return None

    def get_company_ceo(self, ticker_symbol: str) -> Optional[str]:
        """Fetches the name of the CEO for the company associated with the ticker_symbol."""
        try:
            stock = yf.Ticker(ticker_symbol.upper())
            info = stock.info

            # Look for CEO in various possible fields
            ceo = None
            for field in ['companyOfficers', 'officers']:
                if field in info:
                    officers = info[field]
                    if isinstance(officers, list):
                        for officer in officers:
                            if isinstance(officer, dict):
                                title = officer.get('title', '').lower()
                                if 'ceo' in title or 'chief executive' in title:
                                    ceo = officer.get('name')
                                    break

            # Fallback to general company info
            if not ceo and 'longBusinessSummary' in info:
                ceo = None

            return ceo
        except Exception as e:
            print(f"Error fetching CEO info: {e}")
            return None


    def find_ticker_symbol(self, company_name: str) -> Optional[str]:
        """Tries to identify the stock ticker symbol for a given company_name."""
        try:
            # Use yfinance Lookup to search for the company
            lookup = yf.Lookup(company_name)

            stock_results = lookup.get_stock(count=5)

            if not stock_results.empty:
                return stock_results.index[0]

            # If no stocks found, try all instruments
            all_results = lookup.get_all(count=5)

            if not all_results.empty:
                return all_results.index[0]

        except Exception as e:
            print(f"Error searching for ticker: {e}")

        return None


    def ask_user_for_clarification(self, question_to_user: str) -> str:
        """Poses the question_to_user to the actual user and returns their typed response."""
        print(f"\nAgent needs clarification: {question_to_user}")
        response = input("Your response: ")
        return response

    def create_tool_definitions(self) -> List[Dict[str, Any]]:
        """Creates OpenAI function calling definitions for the tools."""
        return [
            {
                "type": "function",
                "function": {
                    "name": "get_stock_price",
                    "description": "Fetches the current stock price for the given ticker symbol",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker_symbol": {
                                "type": "string",
                                "description": "The stock ticker symbol (e.g., 'AAPL', 'MSFT')"
                            }
                        },
                        "required": ["ticker_symbol"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_company_ceo",
                    "description": "Fetches the name of the CEO for the company associated with the ticker symbol",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "ticker_symbol": {
                                "type": "string",
                                "description": "The stock ticker symbol"
                            }
                        },
                        "required": ["ticker_symbol"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "find_ticker_symbol",
                    "description": "Tries to identify the stock ticker symbol for a given company name",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "company_name": {
                                "type": "string",
                                "description": "The name of the company"
                            }
                        },
                        "required": ["company_name"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "ask_user_for_clarification",
                    "description": "Poses a question to the user and returns their response",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "question_to_user": {
                                "type": "string",
                                "description": "The question to ask the user"
                            }
                        },
                        "required": ["question_to_user"]
                    }
                }
            }
        ]

    def execute_tool(self, tool_name: str, arguments: Dict[str, Any]) -> Any:
        """Executes the specified tool with given arguments."""
        if tool_name == "get_stock_price":
            return self.get_stock_price(arguments["ticker_symbol"])
        elif tool_name == "get_company_ceo":
            return self.get_company_ceo(arguments["ticker_symbol"])
        elif tool_name == "find_ticker_symbol":
            return self.find_ticker_symbol(arguments["company_name"])
        elif tool_name == "ask_user_for_clarification":
            return self.ask_user_for_clarification(arguments["question_to_user"])
        else:
            return None

    def process_user_query(self, user_query: str) -> str:
        """Processes a user query using the OpenAI API with function calling."""
        # Add user message to conversation history
        self.conversation_history.append({"role": "user", "content": user_query})

        system_prompt = """You are a helpful stock information assistant. You have access to tools that can:
                        1. Get current stock prices
                        2. Find company CEOs
                        3. Find ticker symbols for company names
                        4. Ask users for clarification when needed

                        Use these tools to help answer user questions about stocks and companies. If information is ambiguous, ask for clarification."""

        while True:
            messages = [
                {"role": "system", "content": system_prompt},
                *self.conversation_history
            ]

            # Call OpenAI API with function calling
            response = self.client.chat.completions.create(
                model="gpt-4-turbo-preview",
                messages=messages,
                tools=self.create_tool_definitions(),
                tool_choice="auto"
            )

            response_message = response.choices[0].message

            # If no tool calls, we're done
            if not response_message.tool_calls:
                self.conversation_history.append({"role": "assistant", "content": response_message.content})
                return response_message.content

            # Execute the first tool call
            tool_call = response_message.tool_calls[0]
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            print(f"\nExecuting tool: {function_name} with args: {function_args}")

            # Execute the tool
            result = self.execute_tool(function_name, function_args)

            # Add the assistant's message with tool calls to history
            self.conversation_history.append({
                "role": "assistant",
                "content": None,
                "tool_calls": [{
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": function_name,
                        "arguments": json.dumps(function_args)
                    }
                }]
            })

            # Add tool result to history
            self.conversation_history.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(result) if result is not None else "No result found"
            })

    def chat(self):
        """Interactive chat loop."""
        print("Stock Information Agent")
        print("Ask me about stock prices, company CEOs, or any stock-related questions!")
        print("Type 'quit' to exit.\n")

        while True:
            user_input = input("You: ")

            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("Goodbye!")
                break

            try:
                response = self.process_user_query(user_input)
                print(f"\nAgent: {response}\n")
            except Exception as e:
                print(f"\nError: {e}\n")

if __name__ == "__main__":
    agent = StockInfoAgent()
    agent.chat()

Stock Information Agent
Ask me about stock prices, company CEOs, or any stock-related questions!
Type 'quit' to exit.

You: tell me stock price of microsoft

Executing tool: find_ticker_symbol with args: {'company_name': 'Microsoft'}

Executing tool: get_stock_price with args: {'ticker_symbol': 'MSFT'}

Agent: The current stock price of Microsoft (MSFT) is $497.45 USD.

You: apple stock price and tcs stock price compare both profit of 5 year

Executing tool: find_ticker_symbol with args: {'company_name': 'Apple'}

Executing tool: get_stock_price with args: {'ticker_symbol': 'AAPL'}

Executing tool: find_ticker_symbol with args: {'company_name': 'Tata Consultancy Services'}

Executing tool: get_stock_price with args: {'ticker_symbol': 'TCS.NS'}

Agent: The current stock price of Apple (AAPL) is $201.00 USD, and for Tata Consultancy Services (TCS.NS), it's $3441.80 USD.

Regarding the comparison of both companies' profits over the past 5 years, I can provide you current stock prices and 

KeyboardInterrupt: Interrupted by user